## Importação de bibliotecas

In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


import time
import pandas as pd
import random
from datetime import datetime


## Importação e tratamento da base

In [103]:
base = pd.read_excel('baseteste.xlsx')
base.columns = base.iloc[2]
base = base[3:]
base = base.reset_index(drop=True)

# display(base)
contatados = pd.DataFrame(columns = ['NOME NO CELULAR', 'TELEFONE', 'STATUS WHATSAPP'])
status = base.loc[0,'STATUS WHATSAPP']
display(base)

True


2,CPF / CNPJ,TELEFONES,EMAIL,NOME NO CELULAR,NOME,STATUS WHATSAPP,STATUS EMAIL
0,NaN,"55415865, 4254854445, 11982662870",NaN,NaN,sergio pj,NaN,NaN
1,NaN,"43215432, 11951496374",NaN,NaN,Yasminzinha Mini,NaN,NaN
2,NaN,"23456789, 11981317092",NaN,NaN,Rog gério,NaN,NaN


## Abertura de navegador

In [105]:
servico = Service(version= '122')
caminho_perfil = r'C:\Users\serci\AppData\Local\Google\Chrome\User Data\Profile Selenium'
options = webdriver.ChromeOptions()
options.add_argument(f'user-data-dir={caminho_perfil}')

navegador = webdriver.Chrome(service=servico, options = options)
navegador.get('https://web.whatsapp.com/')
while len(navegador.find_elements(By.CLASS_NAME, 'selectable-text')) == 0:
    time.sleep(1)

segue


## Definição de funções

In [106]:



def procura_contato(nome, telefone):
    campo_de_busca = navegador.find_element(By.CLASS_NAME, 'selectable-text')
    campo_de_busca.send_keys(Keys.CONTROL, 'a')
    campo_de_busca.send_keys(telefone)
    time.sleep(1)
    contato = navegador.find_elements(By.CLASS_NAME, '_21S-L')
    
    mensagem = f'''
Cliente {nome}, somos da CAIXA - Agência Santo Eduardo do Embu, tudo bem?
#quebra#
Você que é beneficiário do BPC pelo INSS, faça o desbloqueio do seu benefício no INSS e aproveite para fazer seu crédito consignado CAIXA com as menores taxas do  mercado.     
#quebra#                                    
Além disso, você pode ter um cartão de crédito CAIXA sem anuidade e com juros reduzidos.
#quebra#
Caso tenha interesse, responda esta mensagem ou compareça diretamente na agência sem pegar fila procurando por nossos gerentes Níger ou Aldecir.
#quebra#
Atenciosamente CAIXA - Agência Santo Eduardo do Embu - Av. João Paulo II, 1344 - Jardim Santa Tereza - Embu das Artes - SP 
#quebra#
Lembrando que em breve estaremos novamente no bairro do Santo Eduardo, próximo à Drogaria São Paulo.
'''
    if len(contato)> 0:
        contato[0].click()
        envia_mensagem(mensagem)
        retorna_status('enviado, ')
        return
    else:
        retorna_status('tel sem whats,')

def horario():
    # Obtenha o horário atual
    horario_atual = datetime.now()
    # Imprima o horário atual
    return horario_atual.strftime('%H:%M')

def envia_mensagem(texto):
    secciona_mensagem = texto.split('#quebra#')
    for pedaco in secciona_mensagem:
        navegador.find_element(By.CLASS_NAME, '_3Uu1_').send_keys(pedaco, Keys.ENTER)
        time.sleep(1.5)   
        time.sleep(random.randrange(1,6,1))
def retorna_status(texto):
    status = base.loc[cliente, 'STATUS WHATSAPP']
    if pd.isna(status):
        status = texto
    else:
        status = status + texto
    base.loc[cliente, 'STATUS WHATSAPP'] = status
    

## Varrer a base enviando mensagens

In [107]:
contador = 0

print(f'inicio em {horario()}')
for cliente in base.index:
    contador += 1
    nome = base.loc[cliente, 'NOME']
    lista_telefones = base.loc[cliente,'TELEFONES'].split(',')
    for telefone in lista_telefones:
        if telefone != '':
            procura_contato(nome, telefone)
    if contador % 5 == 0:
        contador = 0
        base.to_excel('baseteste.xlsx')
print(f'concluido as {horario()}')
base['STATUS WHATSAPP'] = base['STATUS WHATSAPP'].str[:-1]

inicio em 23:46

aqui estou enviando para sergio pj a mensagem no telefone 55415865
deu 1
deu 2
deu 3

aqui estou enviando para sergio pj a mensagem no telefone  4254854445
deu 1
deu 2
deu 3

aqui estou enviando para sergio pj a mensagem no telefone  11982662870
deu 1
deu 2
deu 3

aqui estou enviando para Yasminzinha Mini a mensagem no telefone 43215432
deu 1
deu 2
deu 3

aqui estou enviando para Yasminzinha Mini a mensagem no telefone  11951496374
deu 1
deu 2
deu 3

aqui estou enviando para Rog gério a mensagem no telefone 23456789
deu 1
deu 2
deu 3

aqui estou enviando para Rog gério a mensagem no telefone  11981317092
deu 1
deu 2
deu 3
concluido as 23:50


2,CPF / CNPJ,TELEFONES,EMAIL,NOME NO CELULAR,NOME,STATUS WHATSAPP,STATUS EMAIL
0,NaN,"55415865, 4254854445, 11982662870",NaN,NaN,sergio pj,"enviado, enviado, enviado,",NaN
1,NaN,"43215432, 11951496374",NaN,NaN,Yasminzinha Mini,"enviado, enviado,",NaN
2,NaN,"23456789, 11981317092",NaN,NaN,Rog gério,"enviado, enviado,",NaN


## Salva retorno da base

In [ ]:
base.to_excel('baseteste.xlsx')